In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

In [3]:
churn_rate = 0.265
customer_ltv = 2283.30
retention_cost = 12.95
recall = 0.82
precision = 0.80

print(f'churn rate : {churn_rate}')
print(f'customer_ltv : {customer_ltv}')
print(f'retention_cost : {retention_cost}')
print(f'recall : {recall}')
print(f'precision : {precision}')

churn rate : 0.265
customer_ltv : 2283.3
retention_cost : 12.95
recall : 0.82
precision : 0.8


In [4]:
# --- helper for ROI computation ---
def churn_roi_summary(customer_ltv, retention_cost, churn_rate, total_customers,
                      precision, recall):
    actual_churners = int(total_customers * churn_rate)
    actual_nonchurners = total_customers - actual_churners
    tp = int(recall * actual_churners)
    fn = actual_churners - tp
    fp = int(tp * (1 / precision - 1))
    flagged = tp + fp

    campaign_cost = flagged * retention_cost
    revenue_protected = tp * customer_ltv
    revenue_lost = fn * customer_ltv
    net_savings = revenue_protected - campaign_cost

    return {
        "Flagged Customers": flagged,
        "Revenue Protected ($)": revenue_protected,
        "Campaign Cost ($)": campaign_cost,
        "Revenue Lost ($)": revenue_lost,
        "Net Savings ($)": net_savings,
        "Precision": round(precision,3),
        "Recall": round(recall,3)
    }

# --- interactive UI ---

# wide_layout = widgets.Layout(width='400px')  # controls input box width
label_style = {'description_width': 'initial'}  # allow long labels to wrap or fully display

ltv_in = widgets.FloatText(
    value=500,
    description="Customer Lifetime Value ($)",
    # layout=wide_layout,
    style=label_style
)

cost_in = widgets.FloatText(
    value=20,
    description="Retention Cost ($)",
    # layout=wide_layout,
    style=label_style
)

churn_in = widgets.FloatSlider(
    value=0.25, min=0.01, max=0.6, step=0.01,
    description="Churn Rate",
    # layout=wide_layout,
    style=label_style,
    readout_format=".2f"
)

cust_in = widgets.IntText(
    value=10000,
    description="Total Customers",
    # layout=wide_layout,
    style=label_style
)

prec_in = widgets.FloatSlider(
    value=0.9, min=0.1, max=1.0, step=0.01,
    description="Model Precision",
    # layout=wide_layout,
    style=label_style,
    readout_format=".2f"
)

rec_in = widgets.FloatSlider(
    value=0.6, min=0.1, max=1.0, step=0.01,
    description="Model Recall",
    # layout=wide_layout,
    style=label_style,
    readout_format=".2f"
)

out = widgets.Output()

def update_table(_=None):
    with out:
        clear_output()
        res = churn_roi_summary(
            ltv_in.value, cost_in.value, churn_in.value, cust_in.value,
            prec_in.value, rec_in.value
        )
        df = pd.DataFrame([res])
        display(df.style.format({
            "Revenue Protected ($)": "{:,.0f}",
            "Campaign Cost ($)": "{:,.0f}",
            "Revenue Lost ($)": "{:,.0f}",
            "Net Savings ($)": "{:,.0f}"
        }).background_gradient(subset=["Net Savings ($)"], cmap="Greens"))

for w in [ltv_in, cost_in, churn_in, cust_in, prec_in, rec_in]:
    w.observe(update_table, names="value")

# with out:
#   display(widgets.VBox([ltv_in, cost_in, churn_in, cust_in, prec_in, rec_in, out]))
# display(out)

display(widgets.VBox([ltv_in, cost_in, churn_in, cust_in, prec_in, rec_in, out]))
update_table()
